In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras import layers
#from tensorflow.keras.utils import plot_model
from keras.models import Model, load_model, save_model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
#from sklearn.model_selection import train_test_split
#from tensorflow.keras.utils import plot_model

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dataset/sub6_data.csv') 

In [ ]:
df_d = df.to_numpy()
#df_y = df['Label']
#df_X = df.drop(columns=['Label'])
#E_1= df.to_numpy()
#X = df_X.to_numpy()
#y = df_y.to_numpy()
df.head()

,a,b,c,d,e,f,g,h,i,j,k,l,Label
0,-0.000502,0.000012,0.000950,-0.001174,-0.000829,-0.000622,0.000066,3.120000e-05,-0.001509,-0.000296,0.000016,-0.000617,1
1,-0.000733,0.001181,0.000245,-0.001414,-0.000638,-0.000588,0.000028,3.750000e-05,-0.001144,-0.000187,0.000014,-0.000671,1
2,-0.000906,0.002351,-0.000263,-0.001566,-0.000475,-0.000556,-0.000017,2.670000e-05,-0.000779,-0.000225,0.000004,-0.000747,1
3,-0.001026,0.003412,-0.000572,-0.001619,-0.000335,-0.000531,-0.000081,9.190000e-07,-0.000512,-0.000334,-0.000016,-0.000837,1
4,-0.001093,0.004388,-0.000670,-0.001581,-0.000211,-0.000511,-0.000174,-3.610000e-05,-0.000425,-0.000483,-0.000043,-0.000926,1


In [ ]:
def create_dataset(df_d, look_back=1):
	dataX, dataY = [], []
	for i in range(len(df_d)-look_back):
		a = df_d[i:(i+look_back), 0:12]
		dataX.append(a)
		dataY.append(df_d[i + look_back, 12])
	return np.array(dataX), np.array(dataY)

In [ ]:
# split into train and test sets
train_size = int(len(df_d) * 0.9)
test_size = len(df_d) - train_size
train, test = df_d[0:train_size,:], df_d[train_size:len(df_d),:]


#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,  random_state=42)
#train_size = int(len(df) * 0.9)
#test_size = len(df) - train_size
#train, test = df.iloc[0:train_size], df.iloc[train_size:len(df),:]
#print(train.shape, test.shape)

In [ ]:
train.shape, test.shape

((486, 13), (54, 13))

In [ ]:
look_back=17 # timesteps
trainX, trainy = create_dataset(train, look_back)
testX, testy = create_dataset(test, look_back)

In [ ]:
trainX.shape, trainy.shape, testX.shape, testy.shape

((469, 17, 12), (469,), (37, 17, 12), (37,))

In [ ]:

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

#X_train = X_train.reshape(X_train.shape[1], X_train.shape[2], 1)
#X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
#from tensorflow.keras.utils import to_categorical
num_classes = len(np.unique(trainy))
trainy = tf.keras.utils.to_categorical(trainy)
testy = tf.keras.utils.to_categorical(testy)

In [ ]:
def make_model(input_shape):
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=128, kernel_size=3, padding="valid", activation='relu')(input_layer)
    #conv1 = keras.layers.BatchNormalization()(conv1)
    drop1 = keras.layers.Dropout(0.5)(conv1)
    pool1 = keras.layers.MaxPooling1D(pool_size=2)(conv1)
    #drop1 = keras.layers.Dropout(0.5)(pool1)
    flat1 = keras.layers.Flatten()(pool1)

    conv2 = keras.layers.Conv1D(filters=64, kernel_size=5, padding="valid", activation='relu')(input_layer)
    #conv2 = keras.layers.BatchNormalization()(conv2)
    drop2 = keras.layers.Dropout(0.5)(conv2)
    pool2 = keras.layers.MaxPooling1D(pool_size=2)(conv2)
    #drop2 = keras.layers.Dropout(0.5)(pool2)
    flat2 = keras.layers.Flatten()(pool2)

    conv3 = keras.layers.Conv1D(filters=32, kernel_size=11, padding= "valid", activation='relu' )(input_layer)
    #conv3 = keras.layers.BatchNormalization()(conv3)
    drop3 = keras.layers.Dropout(0.5)(conv3)
    pool3 = keras.layers.MaxPooling1D(pool_size=2)(conv3)
   # drop3 = keras.layers.Dropout(0.5)(pool3)
    flat3 = keras.layers.Flatten()(pool3)

    merged = keras.layers.concatenate([flat1, flat2, flat3])
    # interpretation
    dense1 = keras.layers.Dense(100, activation='relu')(merged)
    outputs = keras.layers.Dense(num_classes, activation='softmax')(dense1)
    #model = keras.models.Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
    #return model

    #gap = keras.layers.GlobalAveragePooling1D()(conv3)

    #output_layer = keras.layers.Dense(num_classes, activation="softmax")(gap)

    return keras.models.Model(inputs=input_layer, outputs=outputs)

In [ ]:
model = make_model(input_shape=trainX.shape[1:])

#keras.utils.plot_model(model, to_file='model.png', show_shapes=False, show_layer_names=False, dpi=96 )

In [ ]:
epochs = 500 #25 #5 #500
batch_size = 32

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h6", save_best_only=True, verbose=1, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=80, verbose=1, baseline=0.4),
]
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
history = model.fit(
    trainX,
    trainy,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.2,
    verbose=1,
)

Epoch 1/500
12/12 [==============================] - ETA: 0s - loss: 0.6651 - accuracy: 0.6587
Epoch 1: val_loss improved from inf to 0.55945, saving model to best_model.h6
INFO:tensorflow:Assets written to: best_model.h6/assets
12/12 [==============================] - 2s 161ms/step - loss: 0.6651 - accuracy: 0.6587 - val_loss: 0.5595 - val_accuracy: 0.7660 - lr: 0.0010
Epoch 2/500
12/12 [==============================] - ETA: 0s - loss: 0.6331 - accuracy: 0.6720
Epoch 2: val_loss did not improve from 0.55945
12/12 [==============================] - 0s 8ms/step - loss: 0.6331 - accuracy: 0.6720 - val_loss: 0.5602 - val_accuracy: 0.7660 - lr: 0.0010
Epoch 3/500
12/12 [==============================] - ETA: 0s - loss: 0.6289 - accuracy: 0.6720
Epoch 3: val_loss did not improve from 0.55945
12/12 [==============================] - 0s 7ms/step - loss: 0.6289 - accuracy: 0.6720 - val_loss: 0.5628 - val_accuracy: 0.7660 - lr: 0.0010
Epoch 4/500
12/12 [==============================] - ETA: 0

In [ ]:
rounded_labels=np.argmax(testy, axis=1)
rounded_labels[1]

0

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

y_pred = np.argmax(model.predict(testX),axis=1)
accuracy = accuracy_score(rounded_labels, y_pred)
precision = precision_score(rounded_labels, y_pred, zero_division=1)
recall = recall_score(rounded_labels, y_pred, zero_division=1)
print('Accuracy: ', accuracy*100)
print('Precision: ', precision*100)
print('Recall: ', recall*100)

Accuracy:  97.2972972972973
Precision:  100.0
Recall:  0.0


In [ ]:
model = keras.models.load_model("best_model.h6")

#test_loss, test_acc = model.evaluate(x_test, y_test)
test_loss, test_acc = model.evaluate(testX, testy, verbose=0)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

Test accuracy 0.8571428656578064
Test loss 0.4707798659801483


In [ ]:
#tf.keras.metrics.Precision(name = 'precision'), tf.keras.metrics.Recall(name = 'recall') in metric